In [15]:
import pandas as pd 
import os

In [16]:
caption_df = pd.read_csv("data/Mname&Captions.csv")
music_folder = 'data/music'

### Task to do :-
1. Handle CSV with Caption
2. Handle Wav file
3. Make a new df with all the data combine
4. 

In [17]:
caption_df = caption_df[:10]

In [18]:
caption_df

,audio,caption
0,drJaSu3AWhQ_0.wav,The low quality recording features a funky blu...
1,k-J2-Ou1Fm8_4.wav,"Single sounds, noises and melodies being playe..."
2,drJaSu3AWhQ_0.wav,The low quality recording features a funky blu...
3,k-J2-Ou1Fm8_4.wav,"Single sounds, noises and melodies being playe..."
4,9jeEfi6nDak_0.wav,This music sounds oriental full of semitones a...
5,yIFP8fkq8GU_1.wav,THis song contains a piano playing a major cho...
6,bt7rDryN7G4_2.wav,This music is an electronic guitar instrumenta...
7,Zxyfhub6nV4_3.wav,This audio holds a church bell ringing loudly ...
8,1tz4xNRRR4M_3.wav,This lullaby features a fruity male vocal talk...
9,w09XinexaIY_1.wav,The low quality recording features synchronize...


In [19]:
new_data =pd.DataFrame()

In [20]:
import librosa

In [21]:
# Define the function to preprocess audio files
def add_music(file_path):
    y, sr = librosa.load(file_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    return mfccs.T# Transpose to have time steps as rows

In [22]:
new_data["music"] = caption_df["audio"].apply(lambda x: add_music(os.path.join(music_folder, x)))

In [23]:
new_data

,music
0,"[[-77.95577, 57.150143, 12.284519, 13.885426, ..."
1,"[[-57.70024, 79.12726, 15.739094, -3.791172, 2..."
2,"[[-77.95577, 57.150143, 12.284519, 13.885426, ..."
3,"[[-57.70024, 79.12726, 15.739094, -3.791172, 2..."
4,"[[-70.58194, 65.42238, 15.194408, 10.895489, 3..."
5,"[[-39.163975, 87.05516, 8.311457, -9.417724, 8..."
6,"[[-62.635796, 74.84204, 19.518227, 8.522663, 3..."
7,"[[-50.534317, 91.87187, 33.89235, 17.628952, 3..."
8,"[[-67.899994, 67.164185, 12.784273, 5.8597994,..."
9,"[[-62.284046, 73.64278, 17.52047, 8.26719, 30...."


In [24]:
import torch

In [25]:
torch.cuda.is_available()

True

In [26]:
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from collections import Counter
import string

In [29]:
# Text cleaning and tokenization
def preprocess_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text.split()

# Build vocabulary
tokenized_texts = [preprocess_text(text) for text in caption_df['caption']]
vocab = Counter(token for text in tokenized_texts for token in text)
vocab = {word: idx for idx, (word, _) in enumerate(vocab.items(), start=1)}  # 1 for padding
reverse_vocab = {idx: word for word, idx in vocab.items()}

# Convert text to numerical indices
def text_to_indices(text):
    return [vocab.get(token, 0) for token in text]  # 0 is for unknown words

indexed_texts = [text_to_indices(text) for text in tokenized_texts]

# Pad sequences
max_length = max(len(seq) for seq in indexed_texts)
padded_sequences = pad_sequence([torch.tensor(seq) for seq in indexed_texts], batch_first=True, padding_value=0)
